# **<font color='red'>Part 1: Error Checking</font>**

In [ ]:
def checkErrors(regex):
  parCount = 0
  sqrCount = 0

  for i, char in enumerate(regex):
    if char == '(':
      parCount += 1
    elif char == ')':
      parCount -= 1
      if parCount < 0:
        return False
    elif char == '[':
      if ((i+1 == len(regex)) or (not regex[i+1].isalnum())):
        return False
      sqrCount += 1
    elif char == ']':
      sqrCount -= 1
      if sqrCount < 0:
        return False
    elif char == '-':
      if (not regex[i-1].isalnum()) or ((i+1 == len(regex)) or (not regex[i+1].isalnum())) or (ord(regex[i-1]) > ord(regex[i+1])):
        return False
    elif char in ['*', '?', '+']:
      if ((i+1 != len(regex)) and (regex[i+1] in ['*', '?', '+'])):
        return False

  if (sqrCount != 0) or (parCount != 0):
    return False

  return True

# **<font color='red'>Part 2: Regex To NFA</font>**

## **1- Infix to Postfix**

### Preprocessing

In [ ]:
class rng:

  def __init__(self, txt):
    self.txt = txt

  def showTxt(self):
    return self.txt

In [ ]:
def concat(infix):

  new_infix = ""

  for i in range(len(infix) - 1):
    new_infix += infix[i]

    if (infix[i] in ['*','+','?',']',')']  and infix[i+1] not in ['*','+','?',']', '|', ')']):
      new_infix += '•'

    elif ((infix[i].isalnum() or infix[i] == '.') and (infix[i+1].isalnum() or infix[i] == '.' or infix[i+1] in ['(','['])):
       new_infix += '•'

  new_infix += infix[-1]
  return new_infix

In [ ]:
def tokenizeRngs(infix):

  new_infix = list()
  rngString = ""
  rngFlag = False

  for char in infix:

    if char == '[':
      rngFlag = True
      continue

    elif char == ']':
      rngObj = rng(rngString)
      new_infix.append(rngObj)
      rngFlag = False
      rngString = ""
      continue

    if rngFlag:
      rngString += char

    else:
      new_infix.append(char)

  return new_infix

### Shunting Yard


In [ ]:
precedence = {'*': 5, '+': 4, '?': 3, '•': 2, '|': 1, '(': 0}

In [ ]:
def shuntingYard(infix):

  #Preprocessing Steps
  infix = concat(infix)
  infix = tokenizeRngs(infix)
  #Shunting Yard Algorithm
  stack = list()
  postfix = list()

  for char in infix:

    if char == '(':
      stack.append(char)

    elif char == ')':
      while (stack[-1] != '('):
        postfix.append(stack.pop())
      stack.pop()

    elif char in precedence.keys():
      while (stack and (precedence[char] <= precedence[stack[-1]])):
        postfix.append(stack.pop())
      stack.append(char)

    else:
      postfix.append(char)

  while stack:
    postfix.append(stack.pop())

  return postfix

## **2- Postfix to NFA**

### Graph OOP

In [ ]:
class edge:
  def __init__(self, edge_to, value):
    self.edge_to = edge_to
    self.value = value


class state:
  def __init__(self, name, id):
    self.name = name
    self.id = id
    self.edges = list()

class nfa:
  def __init__(self, starting, terminating):
    self.starting = starting
    self.terminating = terminating
    self.states = list()

class nfaOps:
  def buildNFA(self, edgeValue, stateNum):

    state1 = state('S' + str(stateNum), stateNum)
    state2 = state('S' + str(stateNum + 1), stateNum + 1)
    state1.edges.append(edge(state2, edgeValue))
    newNFA = nfa(state1, state2)
    newNFA.states.extend([state1, state2])
    return newNFA

  def concatNFAs(self, nfa1, nfa2):

    nfa1.terminating.edges.append(edge(nfa2.starting, 'ε'))
    newNFA = nfa(nfa1.starting, nfa2.terminating)
    newNFA.states.extend(nfa1.states + nfa2.states)
    return newNFA

  def orNFAS(self, nfa1, nfa2, stateNum):

    state1 = state('S' + str(stateNum), stateNum)
    state2 = state('S' + str(stateNum + 1), stateNum + 1)
    state1.edges.append(edge(nfa1.starting, 'ε'))
    state1.edges.append(edge(nfa2.starting, 'ε'))
    nfa1.terminating.edges.append(edge(state2, 'ε'))
    nfa2.terminating.edges.append(edge(state2, 'ε'))
    newNFA = nfa(state1, state2)
    newNFA.states.extend([state1, state2])
    newNFA.states.extend(nfa1.states + nfa2.states)
    return newNFA

  def zeroOrMoreNFA(self, nfa1, stateNum):

    state1 = state('S' + str(stateNum), stateNum)
    state2 = state('S' + str(stateNum + 1), stateNum + 1)
    state1.edges.append(edge(nfa1.starting, 'ε'))
    state1.edges.append(edge(state2, 'ε'))
    nfa1.terminating.edges.append(edge(state2, 'ε'))
    nfa1.terminating.edges.append(edge(nfa1.starting, 'ε'))
    newNFA = nfa(state1, state2)
    newNFA.states.extend([state1, state2])
    newNFA.states.extend(nfa1.states)
    return newNFA

  def oneOrMoreNFA(self, nfa1, stateNum):

    state1 = state('S' + str(stateNum), stateNum)
    state2 = state('S' + str(stateNum + 1), stateNum + 1)
    state1.edges.append(edge(nfa1.starting, 'ε'))
    nfa1.terminating.edges.append(edge(state2, 'ε'))
    nfa1.terminating.edges.append(edge(nfa1.starting, 'ε'))
    newNFA = nfa(state1, state2)
    newNFA.states.extend([state1, state2])
    newNFA.states.extend(nfa1.states)
    return newNFA

  def zeroOrOneNFA(self, nfa1, stateNum):

    state1 = state('S' + str(stateNum), stateNum)
    state2 = state('S' + str(stateNum + 1), stateNum + 1)
    state1.edges.append(edge(nfa1.starting, 'ε'))
    state1.edges.append(edge(state2, 'ε'))
    nfa1.terminating.edges.append(edge(state2, 'ε'))
    newNFA = nfa(state1, state2)
    newNFA.states.extend([state1, state2])
    newNFA.states.extend(nfa1.states)
    return newNFA

### Constructing Graph

In [ ]:
def postfixToNFA(postfix):

    stack = list()
    id = 0
    ops = nfaOps()

    for char in postfix:

      if(char == '•'):
        nfa1 = stack.pop()
        nfa2 = stack.pop()
        newNFA = ops.concatNFAs(nfa2, nfa1)
        stack.append(newNFA)

      elif(char == '|'):
        nfa1 = stack.pop()
        nfa2 = stack.pop()
        newNFA = ops.orNFAS(nfa2, nfa1, id)
        stack.append(newNFA)
        id += 2

      elif(char == '*'):
        nfa1 = stack.pop()
        newNFA = ops.zeroOrMoreNFA(nfa1, id)
        stack.append(newNFA)
        id += 2

      elif(char == '+'):
        nfa1 = stack.pop()
        newNFA = ops.oneOrMoreNFA(nfa1, id)
        stack.append(newNFA)
        id += 2

      elif(char == '?'):
        nfa1 = stack.pop()
        newNFA = ops.zeroOrOneNFA(nfa1, id)
        stack.append(newNFA)
        id += 2

      else:
        newNFA = ops.buildNFA(char if isinstance(char,str) else char.showTxt().replace('•', ''), id)
        stack.append(newNFA)
        id += 2

    newNFA = stack[-1]
    return newNFA

### Generating JSON file

In [ ]:
import json

def nfaJSON(NFA):

  graph = dict()
  NFA.states.sort(key=lambda x: x.id)
  graph["startingState"] = NFA.starting.name

  for state in NFA.states:

    graph[state.name] = {"isTerminatingState": True if state == NFA.terminating else False}

    for edge in state.edges:

      if edge.value in graph[state.name]:
        graph[state.name][edge.value] = list() + [graph[state.name][edge.value]] + [edge.edge_to.name]
      else:
        graph[state.name][edge.value] = edge.edge_to.name

  with open("NFA.json", "w") as outfile:
    json.dump(graph, outfile, indent= '\t', ensure_ascii=False)

  return graph

### Wrapper Function

In [ ]:
def regexToNFA(regex):
  out = shuntingYard(regex)
  NFA = postfixToNFA(out)
  NFA = nfaJSON(NFA)
  return NFA

# **<font color='red'>Part 2: NFA To Minimized DFA</font>**

## **1- NFA To DFA**

### Some utility functions for range handling

In [ ]:
def expand_alphanumeric_range(input_str):
  expanded_str = ''
  i = 0
  while i < len(input_str):
      if input_str[i] == '.':
          expanded_str += ''.join(chr(c) for c in range(ord('0'), ord('9') + 1))
          expanded_str += ''.join(chr(c) for c in range(ord('A'), ord('Z') + 1))
          expanded_str += ''.join(chr(c) for c in range(ord('a'), ord('z') + 1))
          i += 1
      elif i + 2 < len(input_str) and input_str[i + 1] == '-':
          start_char = input_str[i]
          end_char = input_str[i + 2]
          expanded_str += ''.join(chr(c) for c in range(ord(start_char), ord(end_char) + 1))
          i += 3
      else:
          expanded_str += input_str[i]
          i += 1
  expanded_str = ''.join(sorted(expanded_str))
  return expanded_str

def str_intersections(ranges_list):
  new_ranges = list()
  new_possible_edges = set()
  for i in range(0, len(ranges_list)):
    if len(ranges_list[i]) > 1:
      new_str = ranges_list[i]
      lst = list()
      for edge in new_possible_edges:
        k = 0
        cut_flag = False
        cut_string = ""
        for j in range(len(ranges_list[i])):
          if ranges_list[i][j] == edge[k]:
            cut_string += edge[k]
            k += 1

          elif (ranges_list[i][j] != edge[k]) and (len(cut_string) > 0):
            cut_flag = True

          if (cut_flag) or (((j == len(ranges_list[i]) - 1) or (k == len(edge))) and (len(cut_string) > 0)):
            new_str = new_str.replace(cut_string, '')
            lst.append(cut_string)

            for m in range(len(new_ranges)):
              for n in range(len(new_ranges[m])):
                if (cut_string in new_ranges[m][n]) and (len(new_ranges[m][n]) > len(cut_string)):
                  new_ranges[m][n] = new_ranges[m][n].replace(cut_string, '')
                  new_ranges[m].append(cut_string)

            cut_flag = False
            cut_string = ""
            if not cut_flag:
              break

      lst.append(new_str)
      new_possible_edges.update(lst)
      new_ranges.append(lst)
    else:
      new_ranges.append([ranges_list[i]])
      new_possible_edges.add(ranges_list[i])

  return new_ranges

def group_consecutive_chars(input_str):
  result = []
  i = 0
  while i < len(input_str):
      start = input_str[i]
      end = start
      while i + 1 < len(input_str) and ord(input_str[i + 1]) == ord(input_str[i]) + 1:
          end = input_str[i + 1]
          i += 1
      if start != end:
          result.append(f"{start}-{end}")
      else:
          result.append(start)
      i += 1
  result_str = ''.join(result)
  if result_str == "a-zA-Z0-9":
      return "."
  return result_str

def handle_intersecting_ranges(ranges_list):
  expanded_ranges = [expand_alphanumeric_range(rng) for rng in ranges_list]
  edges_mapping = dict(zip(ranges_list, expanded_ranges))
  edges_mapping = dict(sorted(edges_mapping.items(), key=lambda item: len(item[1])))
  edges_expansion = str_intersections(list(edges_mapping.values()))
  new_possible_edges = set()
  for i in range(len(edges_expansion)):
    for j in range(len(edges_expansion[i])):
      if len(edges_expansion[i][j]) > 1:
        edges_expansion[i][j] = group_consecutive_chars(edges_expansion[i][j])
        new_possible_edges.add(edges_expansion[i][j])
      else:
        new_possible_edges.add(edges_expansion[i][j])
  edges_mapping = dict(zip(edges_mapping.keys(), edges_expansion))

  return edges_mapping, new_possible_edges

### Algorithm

In [ ]:
import copy

def nfaToDfa(nfaFilePath = "NFA.json", nfa = None):

  #Load JSON file
  if not nfa:
    with open(nfaFilePath) as json_file:
      NFA = json.load(json_file)
  else:
    NFA = nfa

  #Set the starting state of DFA
  queue = [{NFA['startingState']}]
  termFlag = False
  if 'ε' in NFA[NFA['startingState']].keys():
    epsEdges = list(NFA[NFA['startingState']]['ε']) if isinstance(NFA[NFA['startingState']]['ε'], list) else [NFA[NFA['startingState']]['ε']]
    queue[0].update(epsEdges)
    while epsEdges:
      nextEdge = epsEdges.pop(0)
      if NFA[nextEdge]["isTerminatingState"]:
        termFlag = True
      if 'ε' in NFA[nextEdge].keys():
        newLst = list(NFA[nextEdge]['ε']) if isinstance(NFA[nextEdge]['ε'], list) else [NFA[nextEdge]['ε']]
        queue[0].update(newLst)
        epsEdges.extend(newLst)

  #Set the graph of DFA
  DFAStates = {str(queue[0]): 'S0'}
  id = 1
  graph = {"startingState": 'S0'}
  graph['S0'] = {"isTerminatingState": termFlag}

  #Start of Conversion
  while queue:
    currentState = queue.pop(0)

    #Get all possible edge transitions
    possibleEdges = [key for state in currentState for key in list(NFA[state].keys())[1:]]
    possibleEdges = set(possibleEdges)
    possibleEdges.discard('ε')

    #Fix edges with ranges
    edgesMapping, possibleEdges = handle_intersecting_ranges(possibleEdges)

    #Create new local NFA with new edge transitions
    NFA_copy = copy.deepcopy(NFA)
    for edge in edgesMapping.keys():
      for state in currentState:
        if edge in NFA_copy[state].keys():
          if len(edgesMapping[edge]) > 1:
            for newEdge in edgesMapping[edge]:
              NFA_copy[state][newEdge] = NFA_copy[state][edge]
          else:
            if edgesMapping[edge][0] != edge:
              NFA_copy[state][edgesMapping[edge][0]] = NFA_copy[state][edge]
              del NFA_copy[state][edge]
            break
          del NFA_copy[state][edge]

    #Iterate over new edges and new NFA
    for edge in possibleEdges:
      newState = set()
      termFlag = False
      for state in currentState:
        if edge in NFA_copy[state].keys():
          newState.add(NFA_copy[state][edge])
          epsEdges = [NFA_copy[state][edge]]
          while epsEdges:
            nextEdge = epsEdges.pop(0)
            if NFA_copy[nextEdge]["isTerminatingState"]:
              termFlag = True
            if 'ε' in NFA_copy[nextEdge].keys():
              newLst = list(NFA_copy[nextEdge]['ε']) if isinstance(NFA_copy[nextEdge]['ε'], list) else [NFA_copy[nextEdge]['ε']]
              newState.update(newLst)
              epsEdges.extend(newLst)

      if (newState and newState not in queue and str(newState) not in DFAStates.keys()):
        queue.append(newState)

      if (newState and str(newState) not in DFAStates.keys()):
        DFAStates[str(newState)] = 'S' + str(id)
        id += 1
        graph[DFAStates[str(newState)]] = {"isTerminatingState": termFlag}
        graph[DFAStates[str(currentState)]][edge] = DFAStates[str(newState)]

      elif(newState):
        graph[DFAStates[str(currentState)]][edge] = DFAStates[str(newState)]

  with open("DFA.json", "w") as outfile:
    json.dump(graph, outfile, indent= '\t')

  return graph

## **2- Minimize DFA**

In [ ]:
def MinimizeDFA(nfaFilePath = "NFA.json", nfa = None, dfa = None):

  #Get DFA
  DFA = (nfaToDfa(nfaFilePath= nfaFilePath) if not nfa else nfaToDfa(nfa= nfa)) if not dfa else dfa

  #Get all possible edge transitions
  states = list(DFA.values())[1:]
  possibleEdges = [key for edge in states for key in list(edge.keys())[1:]]
  possibleEdges = set(possibleEdges)

  #Group states into terminating and non-terminating groups
  stateGroups = {0: [], 1: []}
  groupMapping = dict()
  for key,value in list(DFA.items())[1:]:
    if not value['isTerminatingState']:
      stateGroups[0].append(key)
      groupMapping[key] = 0
    else:
      stateGroups[1].append(key)
      groupMapping[key] = 1
  queue = [(key, value) for key,value in stateGroups.items()]

  #Process each group:
  id = 1
  while queue:
    i, stateGroup = queue.pop(0)

    if len(stateGroup) > 1:
      group = [x for x in stateGroup]

      for edge in possibleEdges:
        transitionGroup = dict()

        for state in group:
          if edge in DFA[state].keys():
            transitionGroup[state] = groupMapping[DFA[state][edge]]
          else:
            transitionGroup[state] = ""

        #Compare by the transition group of the first state
        cmpGroup = list(transitionGroup.values())[0]

        for transitionI in transitionGroup.keys():
          if (transitionGroup[transitionI] != cmpGroup) and (transitionI in group):
            id += 1
            groupMapping[transitionI] = id
            newGroup = list()
            newGroup.append(transitionI)
            group.remove(transitionI)
            stateGroups[i].remove(transitionI)
            for transitionJ in group:
              if transitionGroup[transitionJ] == transitionGroup[transitionI]:
                groupMapping[transitionJ] = id
                newGroup.append(transitionJ)
                stateGroups[i].remove(transitionJ)

            group = [x for x in stateGroups[i]]
            queue.append((id, newGroup))
            stateGroups[id] = newGroup

        if stateGroups[i] != stateGroup:
          queue.append((i, group))

  #Generating graph and JSON file:
  graph = dict()

  for index, group in stateGroups.items():
    if DFA['startingState'] in group:
      graph['startingState'] = 'S' + str(index)

  for index, group in stateGroups.items():
    if group:
      graph['S' + str(index)] = {'isTerminatingState': DFA[group[0]]['isTerminatingState']}

      for edge in list(DFA[group[0]].keys())[1:]:
        graph['S' + str(index)][edge] = 'S' + str(groupMapping[DFA[group[0]][edge]])

  with open("MinimizedDFA.json", "w") as outfile:
    json.dump(graph, outfile, indent= '\t')

  return DFA, graph

# **<font color='red'>Part 3: Bringing It All Together (Regex To Minimized DFA)</font>**

## **Visualization**

In [ ]:
import pydot

def visualGraph(graphName, pathToJSON = None, graph= None):

  #Load JSON file or dictionary graph
  if not graph:
    with open(pathToJSON) as json_file:
      graph = json.load(json_file)
  else:
    graph = graph

  #Preparing visual graph
  visual = pydot.Dot(graphName, graph_type= "digraph")

  #Create all nodes
  for state in graph.keys():
    if state == 'startingState':
      visual.add_node(pydot.Node('Starting Node', style= "invis"))
      visual.add_node(pydot.Node(graph['startingState'], label= graph[state], shape= "circle"))
      visual.add_edge(pydot.Edge('Starting Node', graph['startingState'], label="Start", style= "solid", color="red"))
    elif state != graph['startingState']:
      visual.add_node(pydot.Node(state, label= state, shape= "doublecircle" if graph[state]['isTerminatingState'] else "circle"))

  #Create all edges
  for state in list(graph.keys())[1:]:
    for edge in list(graph[state].keys())[1:]:
      if isinstance(graph[state][edge], str):
        visual.add_edge(pydot.Edge(state, graph[state][edge], label= edge, style= "solid"))
      else:
        for epsEdge in graph[state][edge]:
          visual.add_edge(pydot.Edge(state, epsEdge, label= edge, style= "solid"))

  #Save into image
  visual.write_png(graphName + ".png")

## **Wrapper Function**

In [ ]:
def regextoMinimizedDFA(regex):
  #Check for errors:
  if not checkErrors(regex):
    print("Please Check The Validity Of The Entered Regex")
    return False

  #NFA
  NFA = regexToNFA(regex)
  fileNFA = visualGraph("NFA", graph= NFA)

  #DFA & Minimized DFA
  DFA, minimizedDFA = MinimizeDFA(nfa= NFA)
  fileDFA = visualGraph("DFA", graph= DFA)
  fileMinimizeNFA = visualGraph("MinimizedDFA", graph= minimizedDFA)
  return NFA, DFA, minimizedDFA

## **Input from user**

In [ ]:
regex = input("Please Enter a Regex: ")
regextoMinimizedDFA(regex)

Please Enter a Regex: AB(A|B)*AB


({'startingState': 'S0',
  'S0': {'isTerminatingState': False, 'A': 'S1'},
  'S1': {'isTerminatingState': False, 'ε': 'S2'},
  'S2': {'isTerminatingState': False, 'B': 'S3'},
  'S3': {'isTerminatingState': False, 'ε': 'S10'},
  'S4': {'isTerminatingState': False, 'A': 'S5'},
  'S5': {'isTerminatingState': False, 'ε': 'S9'},
  'S6': {'isTerminatingState': False, 'B': 'S7'},
  'S7': {'isTerminatingState': False, 'ε': 'S9'},
  'S8': {'isTerminatingState': False, 'ε': ['S4', 'S6']},
  'S9': {'isTerminatingState': False, 'ε': ['S11', 'S8']},
  'S10': {'isTerminatingState': False, 'ε': ['S8', 'S11']},
  'S11': {'isTerminatingState': False, 'ε': 'S12'},
  'S12': {'isTerminatingState': False, 'A': 'S13'},
  'S13': {'isTerminatingState': False, 'ε': 'S14'},
  'S14': {'isTerminatingState': False, 'B': 'S15'},
  'S15': {'isTerminatingState': True}},
 {'startingState': 'S0',
  'S0': {'isTerminatingState': False, 'A': 'S1'},
  'S1': {'isTerminatingState': False, 'B': 'S2'},
  'S2': {'isTerminatingS